# My own testing

In [1]:
!sh pip install -r requirements.txt

/Users/allennikka/repos/word-game-solvers/.venv/bin/pip: line 3: import: command not found
/Users/allennikka/repos/word-game-solvers/.venv/bin/pip: line 4: import: command not found
/Users/allennikka/repos/word-game-solvers/.venv/bin/pip: line 5: from: command not found
/Users/allennikka/repos/word-game-solvers/.venv/bin/pip: pip: line 7: syntax error near unexpected token `('
/Users/allennikka/repos/word-game-solvers/.venv/bin/pip: pip: line 7: `    sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])'


In [2]:
# Reload external files
%load_ext autoreload
%autoreload 2

In [8]:
import logging
from tqdm import tqdm_notebook
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
logging.basicConfig(level=logging.INFO)

/Users/allennikka/repos/word-game-solvers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import pickle
def save_embeddings(word_list, embeddings, file_path='embeddings.pkl'):
    with open(file_path, 'wb') as f:
        pickle.dump((word_list, embeddings), f)

# Function to load embeddings from a file
def load_embeddings(file_path='embeddings.pkl'):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

In [9]:
def load_words():
    with open('words_alpha.txt') as word_file:
        valid_words = set(word_file.read().split())
        valid_words = list(map(lambda x: x.lower(), valid_words))
    return valid_words

word_list = load_words()
print(f"Loaded {len(word_list)} words")

Loaded 370103 words


In [10]:

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(word_list, show_progress_bar=True)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Batches: 100%|██████████| 11566/11566 [03:05<00:00, 62.35it/s]


In [14]:
embeddings_file = 'embeddings.pkl'
import os
if os.path.exists(embeddings_file):
    word_list, embeddings = load_embeddings(embeddings_file)
    print("Loaded embeddings from file")
else:
    # Generate embeddings using BERTopic
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(word_list, show_progress_bar=True)
    save_embeddings(word_list, embeddings, embeddings_file)
    print("Generated and saved embeddings")

Loaded embeddings from file


In [15]:
word_embeddings = dict(zip(word_list, embeddings))

In [16]:
def find_similar_words(word, top_n=5):
    if word not in word_embeddings:
        print(f"Word '{word}' not in vocabulary")
        return []
    
    word_emb = word_embeddings[word].reshape(1, -1)
    similarities = cosine_similarity(word_emb, embeddings).flatten()
    similar_indices = similarities.argsort()[-top_n-1:-1][::-1]
    similar_words = [word_list[i] for i in similar_indices]
    
    return similar_words

In [21]:
target_word = 'sneaky'
similar_words = find_similar_words(target_word, top_n=30)
print(f"Words similar to '{target_word}': {similar_words[::-1]}")

Words similar to 'sneaky': ['stealthfully', 'catchy', 'stealthier', 'stealth', 'hacky', 'stealthily', 'stealthlike', 'stealthful', 'tricksy', 'stealthiest', 'disguisedly', 'sneakingness', 'stealthy', 'sneaker', 'sneaksby', 'sneaksman', 'sneaking', 'sneakbox', 'sneakingly', 'sneakered', 'sneaked', 'sneaks', 'sneakishly', 'sneakiness', 'sneakishness', 'sneak', 'sneakier', 'sneakily', 'sneakiest', 'sneakish']
